# Credit Default Prediction Model: Experimental Summary

## Experiment Overview

Evaluated several classification models to predict credit default risk, using fastai's tabular learning framework and custom metrics focused on discrimination capability.

## Metrics Used

We implemented and tracked three key metrics:
- **Accuracy**: Standard classification accuracy
- **AUC Score**: Area under the ROC curve (between 0.5-1.0)
- **Gini Coefficient**: 2*AUC-1 (between 0-1), a common credit risk metric

## Key Experiments and Results

| Experiment | Configuration | Accuracy | AUC | Gini | Best Epoch Performance |
|------------|---------------|----------|-----|------|------------------------|
| 1 | 10 epochs, lr=5e-3, wd=0.1 | 0.763 | 0.701 | 0.401 | Epoch 7: Gini 0.420 |
| 2 | 10 epochs, lr=5e-3, wd=0.01 | 0.743 | 0.684 | 0.367 | Epoch 4: AUC 0.717 |
| 3 | 10 epochs, lr=3e-3, wd=0.01 | 0.738 | 0.664 | 0.328 | Epoch 8: Gini 0.330 |
| 4 | 5 epochs, lr=3e-3, wd=0.01* | 0.736 | 0.662 | 0.324 | Epoch 4: AUC 0.651 |
| 5 | 5 epochs, lr=3e-3, wd=0.05 | 0.764 | 0.697 | 0.394 | Epoch 3: AUC 0.699, Gini 0.399 |
| Ensemble | Models 1+2+3 | 0.759 | 0.697 | 0.394 | N/A |

*with adjusted class weights

## Hyperparameter Impact

| Parameter | Tested Values | Observation |
|-----------|---------------|-------------|
| Weight Decay | 0.01, 0.05, 0.1 | Higher values (0.1) produced better Gini coefficients |
| Learning Rate | 3e-3, 5e-3 | 5e-3 performed better across metrics |
| Class Weights | default, 0.9*default | Reducing minority class weight improved balance |
| Epochs | 5, 7, 10 | 5-7 epochs sufficient; diminishing returns after |

## Model Comparison with Champion

| Statistic | Our Ensemble vs Champion Model |
|-----------|--------------------------------|
| Mean PD Difference | +0.304 higher |
| Mean Score Difference | -249.43 points lower |
| Max PD Difference | +0.572 (APP_ID: 6341) |
| Min PD Difference | +0.028 (APP_ID: 4743) |
| Max Score Difference | -402.00 points (APP_ID: 6341) |
| Min Score Difference | -79.95 points (APP_ID: 4743) |

## Future Work Opportunities

| Area | Potential Improvements |
|------|------------------------|
| Feature Engineering | Domain-specific features, dimensionality reduction |
| Model Architecture | Deeper networks, activation function testing |
| Validation | Cross-validation, Bayesian hyperparameter optimization |
| Calibration | Probability alignment, score scaling |
| Explainability | Feature importance, SHAP values |
| Thresholds | Business-cost based threshold optimization |


# Data preprocess and Features engineering 

In [ ]:
def enhanced_feature_engineering(df):
    "Engineer new features and convert appropriate columns to categorical"
    df = df.copy()
    
    # Log transformations
    df['log_revol_bal'] = np.log1p(df['revol_bal'])
    df['log_annual_inc'] = np.log1p(df['annual_inc'])
    
    # Ratio features
    df['debt_income_ratio'] = df['revol_bal'] / df['annual_inc'].clip(lower=1)
    df['util_acc_ratio'] = df['revol_util'] / df['open_acc'].clip(lower=1)
    
    # Interaction terms
    df['dti_util'] = df['debt_to_income'] * df['revol_util']
    
    # Convert identified columns to categorical
    for col in ['delinq_2yrs', 'inq_last_6mths', 'mths_since_last_record', 'pub_rec']:
        df[f'{col}_cat'] = df[col].astype('category')
    
    return df



In [ ]:
def preprocess_data(df):
    "Preprocess loan application data with appropriate handling of missing values"
    # Make a copy to avoid modifying original
    df = df.copy()
    
    # Calculate missing percentages
    missing_pct = df.isna().sum()/df.shape[0]
    
    # Handle low missing rate columns (<1%)
    low_missing = missing_pct[(missing_pct < 0.01) & (missing_pct > 0)].index
    for col in low_missing:
        mode_val = df[col].mode()[0]
        df[col] = df[col].fillna(mode_val)
    
    # Handle medium missing rate columns (1-40%)
    df['emp_title'] = df['emp_title'].fillna('Unknown')
    df['Notes'] = df['Notes'].fillna('Not shared')
    
    # Handle high missing rate columns (>40%)
    high_missing = missing_pct[missing_pct > 0.40].index
    for col in high_missing:
        df[f'has_{col}'] = df[col].notna().astype(int)
        df[col] = df[col].fillna(0)
    
    return df


In [ ]:
def feature_engineering(df):
    "Engineer new features and clean up existing ones"
    df = df.copy()
    
    # Drop high cardinality and less useful columns
    cols_to_drop = ['emp_title', 'Notes', 'purpose', 'zip_code', 'APP_ID', 'collections_12_mths_ex_med'] 
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
    
    # Convert earliest_cr_line to credit history years
    if 'earliest_cr_line' in df.columns:
        df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
        reference_year = 2008
        df['years_since_first_cr'] = reference_year - df['earliest_cr_line'].dt.year
        df = df.drop(columns=['earliest_cr_line'])
    
    # Handle low cardinality features
    # Convert pub_rec to categorical (very skewed distribution, only 4 values)
    if 'pub_rec' in df.columns:
        df['pub_rec_cat'] = df['pub_rec'].astype('category')
    
    # We keep delinq_2yrs and inq_last_6mths as continuous since they have more values
    # and likely have a somewhat linear relationship with default risk
    
    return df

In [ ]:
def adjust_column_types(cont_cols, cat_cols, move_to_cat=None, move_to_cont=None):
    "Move columns between continuous and categorical lists"
    if move_to_cat:
        cont_cols = [col for col in cont_cols if col not in move_to_cat]
        for col in move_to_cat:
            if col not in cat_cols: cat_cols.append(col)
    
    if move_to_cont:
        cat_cols = [col for col in cat_cols if col not in move_to_cont]
        for col in move_to_cont:
            if col not in cont_cols: cont_cols.append(col)
            
    return cont_cols, cat_cols

# Training

In [ ]:
from utils import *
from fastai.tabular.all import *
# import fastai metric and sklean AUCScore and roc 
from sklearn.metrics import roc_auc_score
from fastai.metrics import *






In [ ]:
cd static

[Errno 2] No such file or directory: 'static'
/app/data/static


In [ ]:
df = pd.read_csv('Assessment Modeling Dataset.csv')
# exp1
# df= preprocess_data(df)
# df= feature_engineering(df)
# cont_cols,cat_cols = cont_cat_split(df, max_card=20, dep_var='TARGET')
# cont_cols, cat_cols = adjust_column_types(cont_cols, cat_cols, move_to_cat=['pub_rec'])


#exp2
df_clean = preprocess_data(df)
df_clean = feature_engineering(df_clean)
df_enhanced = enhanced_feature_engineering(df_clean)

# Get initial column splits
cont_cols, cat_cols = cont_cat_split(df_enhanced, max_card=20, dep_var='TARGET')

# Move columns from continuous to categorical
cols_to_move = ['delinq_2yrs_cat', 'inq_last_6mths_cat', 'mths_since_last_record_cat', 'pub_rec_cat']
cont_cols, cat_cols = adjust_column_types(cont_cols, cat_cols, move_to_cat=cols_to_move)

# Remove original columns from continuous list
cols_to_remove = ['delinq_2yrs', 'inq_last_6mths', 'mths_since_last_record', 'pub_rec']
cont_cols = [col for col in cont_cols if col not in cols_to_remove]
df = df_enhanced.copy()

In [ ]:
to

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[1], line 1
----> 1 to

NameError: name 'to' is not defined


NameError: name 'to' is not defined

In [ ]:
df[cont_cols].nunique()

annual_inc                1590
debt_to_income            2471
mths_since_last_delinq      91
open_acc                    35
revol_bal                 6694
revol_util                1020
total_acc                   73
years_since_first_cr        50
log_revol_bal             6694
log_annual_inc            1590
debt_income_ratio         7703
util_acc_ratio            4886
dti_util                  7718
dtype: int64

In [ ]:
df[cat_cols].isna().sum()

emp_length                     0
home_ownership                 0
verification_status            0
pymnt_plan                     0
purpose_cat                    0
addr_state                     0
initial_list_status            0
mths_since_last_major_derog    0
policy_code                    0
has_mths_since_last_delinq     0
has_mths_since_last_record     0
pub_rec_cat                    0
delinq_2yrs_cat                0
inq_last_6mths_cat             0
mths_since_last_record_cat     0
dtype: int64

In [ ]:
df[cat_cols].nunique()

emp_length                     14
home_ownership                  5
verification_status             3
pymnt_plan                      2
purpose_cat                    27
addr_state                     49
initial_list_status             2
mths_since_last_major_derog     3
policy_code                     5
has_mths_since_last_delinq      2
has_mths_since_last_record      2
pub_rec_cat                     4
delinq_2yrs_cat                10
inq_last_6mths_cat             18
mths_since_last_record_cat     88
dtype: int64

In [ ]:
from fastai.tabular.all import *

# Create train/validation splits
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

# Create TabularPandas
to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                  cat_names=cat_cols,
                  cont_names=cont_cols,
                  y_names='TARGET',
                  splits=splits,
                  y_block=CategoryBlock(),
                  )

dls = to.dataloaders(bs=64)



In [ ]:
dls.show_batch()

,emp_length,home_ownership,verification_status,pymnt_plan,purpose_cat,addr_state,initial_list_status,mths_since_last_major_derog,policy_code,has_mths_since_last_delinq,has_mths_since_last_record,pub_rec_cat,delinq_2yrs_cat,inq_last_6mths_cat,mths_since_last_record_cat,annual_inc,debt_to_income,mths_since_last_delinq,open_acc,revol_bal,revol_util,total_acc,years_since_first_cr,log_revol_bal,log_annual_inc,debt_income_ratio,util_acc_ratio,dti_util,TARGET
0,6,RENT,VERIFIED - income,n,debt consolidation,CA,f,1,PC4,0,0,0.0,0.0,0.0,0.0,31000.001211,14.090000,2.279350e-07,7.0,6458.000307,45.500000,24.0,9.0,8.773230e+00,10.341775,2.083226e-01,6.500000,641.094970,0
1,2,MORTGAGE,not verified,n,debt consolidation,VA,f,1,PC3,0,0,0.0,0.0,1.0,0.0,86003.999576,4.650000,2.279350e-07,16.0,8704.000038,28.899999,42.0,9.0,9.071652e+00,11.362161,1.012046e-01,1.806250,134.385008,0
2,1,RENT,VERIFIED - income source,n,debt consolidation,CA,f,2,PC1,0,0,0.0,0.0,0.0,0.0,19199.998390,13.500000,2.279350e-07,6.0,7082.999931,37.099999,8.0,2.0,8.865594e+00,9.862718,3.689063e-01,6.183333,500.850011,0
3,2,RENT,not verified,n,debt consolidation,CT,f,1,PC2,0,0,0.0,0.0,1.0,0.0,24999.998527,18.049999,2.279350e-07,5.0,9267.999938,47.500000,15.0,7.0,9.134431e+00,10.126671,3.707200e-01,9.500000,857.375005,1
4,5,OWN,not verified,n,debt consolidation,OH,f,1,PC4,1,1,1.0,0.0,2.0,86.0,24000.000537,16.549999,6.200000e+01,4.0,4792.999897,63.099999,17.0,13.0,8.475121e+00,10.085851,1.997083e-01,15.775000,1044.305040,1
5,4,RENT,VERIFIED - income,n,major purchase,TX,f,3,PC3,0,0,0.0,0.0,0.0,0.0,47000.000011,5.970000,2.279350e-07,8.0,8747.999943,33.900001,23.0,10.0,9.076694e+00,10.757924,1.861277e-01,4.237500,202.382983,1
6,10,MORTGAGE,VERIFIED - income source,n,credit card,PA,f,3,PC3,0,0,0.0,0.0,3.0,0.0,71999.999888,16.820000,2.279350e-07,25.0,7401.999638,42.799999,38.0,3.0,8.909640e+00,11.184435,1.028056e-01,1.712000,719.895997,0
7,9,MORTGAGE,VERIFIED - income,n,debt consolidation,SC,f,3,PC1,0,0,0.0,0.0,1.0,0.0,85000.000296,10.090000,2.279350e-07,14.0,12126.000099,34.500000,37.0,22.0,9.403190e+00,11.350418,1.426588e-01,2.464286,348.104997,0
8,9,RENT,VERIFIED - income,n,debt consolidation,TX,f,3,PC5,0,0,0.0,0.0,1.0,0.0,62068.800737,16.160000,2.279350e-07,4.0,-0.000541,47.360001,19.0,9.0,2.581675e-07,11.036015,-5.295631e-09,11.840000,765.337587,0
9,4,RENT,not verified,n,other,CA,f,2,PC2,0,0,0.0,0.0,0.0,0.0,85999.999761,4.790000,2.279350e-07,14.0,28088.999375,31.700001,19.0,13.0,1.024317e+01,11.362114,3.266163e-01,2.264286,151.842997,0


# Metric

In [ ]:
class AUCScore(Metric):
    "Computes ROC AUC score"
    def __init__(self): self.reset()
    
    def reset(self): 
        self.preds,self.targs = [],[]
        
    def accumulate(self, learn):
        pred = learn.pred[:,1] if len(learn.pred.shape)>1 else learn.pred
        self.preds.append(pred)
        self.targs.append(learn.yb[0])
    
    @property
    def value(self):
        preds = torch.cat(self.preds).float().sigmoid()
        targs = torch.cat(self.targs)
        if targs.sum() == 0 or targs.sum() == len(targs): return 0.5
        return roc_auc_score(targs.cpu().numpy(), preds.cpu().numpy())
    
    @property
    def name(self): return "auc"

class GiniCoefficient(Metric):
    "Computes Gini coefficient (2*AUC-1)"
    def __init__(self): self.reset()
    
    def reset(self): 
        self.preds,self.targs = [],[]
        
    def accumulate(self, learn):
        pred = learn.pred[:,1] if len(learn.pred.shape)>1 else learn.pred
        self.preds.append(pred)
        self.targs.append(learn.yb[0])
    
    @property
    def value(self):
        preds = torch.cat(self.preds).float().sigmoid()
        targs = torch.cat(self.targs)
        if targs.sum() == 0 or targs.sum() == len(targs): return 0.0
        return 2 * roc_auc_score(targs.cpu().numpy(), preds.cpu().numpy()) - 1
    
    @property
    def name(self): return "gini"

# Create tabular learner with both metrics
learn = tabular_learner(dls, metrics=[accuracy, AUCScore(), GiniCoefficient()])
learn.loss_func = BCEWithLogitsLossFlat()


In [ ]:
learn

In [ ]:
def find_optimal_lr(dls):
    "Find the optimal learning rate using fastai's LR Finder"
    # Calculate class weights
    counts = dls.train_ds.y.value_counts()
    total = len(dls.train_ds)
    weights = torch.FloatTensor([total/counts[0], total/counts[1]])
    
    # Create learner
    learn = tabular_learner(dls, metrics=[accuracy, AUCScore(), GiniCoefficient()])
    learn.loss_func = CrossEntropyLossFlat(weight=weights)
    
    # Run the learning rate finder
    learn.lr_find()
    
    return learn

In [ ]:
def train_model(dls, epochs=1, lr=1e-2, wd=0.1, metrics=None):
    "Train tabular model with class weights and metrics"
    if metrics is None: metrics = [accuracy, AUCScore(), GiniCoefficient()]
    
    # Calculate class weights
    counts = dls.train_ds.y.value_counts()
    total = len(dls.train_ds)
    weights = torch.FloatTensor([total/counts[0],  0.9 *total/counts[1]])
    print("reduce clss weights" ,weights)
    
    # Create and train model
    learn = tabular_learner(dls, metrics=metrics,lr=lr)
    learn.loss_func = CrossEntropyLossFlat(weight=weights)
    learn.fit_one_cycle(epochs, wd=wd)
    
    return learn


# Experiments

In [ ]:
model = train_model(dls, epochs=10, lr=5e-3, wd=0.1)

reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.685172    0.692187    0.652500  0.602020  0.204040  00:01     
1         0.626640    0.681220    0.701875  0.703296  0.406592  00:01     
2         0.617019    0.662473    0.710625  0.637173  0.274346  00:01     
3         0.584768    0.618681    0.730000  0.678498  0.356995  00:01     
4         0.570455    0.637957    0.740000  0.616021  0.232043  00:01     
5         0.531334    0.668278    0.745000  0.653356  0.306713  00:01     
6         0.484508    0.691316    0.723125  0.670692  0.341383  00:01     
7         0.484830    0.726408    0.726875  0.669043  0.338087  00:01     
8         0.436396    0.718740    0.731875  0.670946  0.341893  00:01     
9         0.419106    0.719462    0.741875  0.669081  0.338161  00:01     


In [ ]:
model = train_model(dls, epochs=10, lr=5e-3, wd=0.01)

reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.681558    0.706821    0.611250  0.561627  0.123253  00:01     
1         0.642229    0.657171    0.665625  0.672418  0.344836  00:01     
2         0.604862    0.612978    0.745625  0.637741  0.275481  00:01     
3         0.565314    0.667196    0.706875  0.654828  0.309656  00:01     
4         0.538620    0.720933    0.765625  0.629802  0.259604  00:01     
5         0.517267    0.685704    0.758750  0.656724  0.313448  00:01     
6         0.498308    0.696284    0.714375  0.666486  0.332972  00:01     
7         0.466765    0.738121    0.732500  0.673787  0.347575  00:01     
8         0.448560    0.759644    0.756250  0.657285  0.314570  00:01     
9         0.424659    0.763045    0.743125  0.657924  0.315847  00:01     


In [ ]:
model = train_model(dls, epochs=10, lr=3e-3, wd=0.01)

reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.686386    0.691322    0.594375  0.530030  0.060060  00:01     
1         0.650289    0.627479    0.676875  0.631790  0.263580  00:01     
2         0.602770    0.609820    0.745000  0.704040  0.408080  00:01     
3         0.558100    0.666668    0.714375  0.627766  0.255533  00:01     
4         0.538220    0.673901    0.730625  0.653305  0.306611  00:01     
5         0.500425    0.720055    0.730000  0.647769  0.295539  00:01     
6         0.497082    0.687992    0.708125  0.665958  0.331915  00:01     
7         0.458898    0.727618    0.709375  0.665788  0.331575  00:01     
8         0.428492    0.734619    0.729375  0.661410  0.322821  00:01     
9         0.422467    0.735374    0.741250  0.663820  0.327640  00:01     


In [ ]:
model = train_model(dls, epochs=5, lr=3e-3, wd=0.01)

reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.681258    0.761254    0.603750  0.603107  0.206215  00:01     
1         0.629167    0.623601    0.729375  0.668228  0.336455  00:01     
2         0.582763    0.628648    0.740625  0.678854  0.357709  00:01     
3         0.547415    0.645700    0.723125  0.684707  0.369413  00:01     
4         0.515910    0.630245    0.758125  0.682990  0.365981  00:01     


In [ ]:
def train_model_deeper(dls, layers=[200,100], ps=[0.2,0.1], epochs=5, lr=3e-3, wd=0.01):
    """Train a deeper tabular model with custom layers and dropout rates"""
    
    # Calculate class weights
    counts = dls.train_ds.y.value_counts()
    total = len(dls.train_ds)
    weights = torch.FloatTensor([total/counts[0], 0.9*total/counts[1]])
    print("Class weights:", weights)
    
    # Create and train model with deeper architecture
    learn = tabular_learner(dls, 
                           layers=layers,  # Define hidden layers
                           ps=ps,          # Define dropout probabilities
                           metrics=[accuracy, AUCScore(), GiniCoefficient()],
                           lr=lr)
    
    learn.loss_func = CrossEntropyLossFlat(weight=weights)
    learn.fit_one_cycle(epochs, wd=wd)
    
    return learn

In [ ]:
model = train_model(dls, epochs=7, lr=5e-3, wd=0.1, metrics=[accuracy, AUCScore(), GiniCoefficient()])

reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.705729    0.769055    0.650000  0.575458  0.150916  00:01     
1         0.628756    0.641804    0.757500  0.681152  0.362304  00:01     
2         0.594742    0.623336    0.730625  0.694484  0.388967  00:01     
3         0.567650    0.645293    0.704375  0.664058  0.328116  00:01     
4         0.537027    0.653974    0.728750  0.689495  0.378990  00:01     
5         0.512876    0.645672    0.719375  0.689101  0.378201  00:01     
6         0.499065    0.643962    0.739375  0.685651  0.371303  00:01     


# Ensemble

In [ ]:
# Train model 1 - our best configuration
model1 = train_model(dls, epochs=7, lr=5e-3, wd=0.1)

# Train model 2 - lower weight decay
model2 = train_model(dls, epochs=7, lr=5e-3, wd=0.01)

# Train model 3 - different learning rate
model3 = train_model(dls, epochs=7, lr=3e-3, wd=0.05)



reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.703916    0.763990    0.631250  0.573429  0.146859  00:01     
1         0.620757    0.670864    0.593750  0.622285  0.244569  00:01     
2         0.582143    0.626732    0.697500  0.681172  0.362344  00:01     
3         0.540791    0.687515    0.728750  0.678168  0.356336  00:01     
4         0.521627    0.695111    0.750625  0.683259  0.366518  00:01     
5         0.506042    0.692024    0.728750  0.678063  0.356125  00:01     
6         0.479794    0.686041    0.743750  0.674093  0.348187  00:01     
reduce clss weights tensor([1.1449, 7.1111])
epoch     train_loss  valid_loss  accuracy  auc       gini      time    
0         0.695147    0.697867    0.624375  0.579964  0.159929  00:01     
1         0.629700    0.650603    0.743750  0.649312  0.298624  00:01     
2         0.591104    0.666104    0.658125  0.659273  0.318546  00:01     
3         0.55

In [ ]:
def ensemble_predictions(models, dl=None):
    """
    Create ensemble predictions by averaging model outputs
    
    Args:
        models: List of trained fastai Learner objects
        dl: Optional dataloader to use for predictions (defaults to validation)
    
    Returns:
        Averaged predictions and targets
    """
    all_preds = []
    targets = None
    
    for i, model in enumerate(models):
        print(f"Getting predictions from model {i+1}/{len(models)}")
        preds, targs = model.get_preds(dl=dl)
        all_preds.append(preds)
        if targets is None:
            targets = targs
    
    # Average predictions
    avg_preds = torch.stack(all_preds).mean(0)
    return avg_preds, targets

In [ ]:
# Get ensemble predictions
avg_preds, targets = ensemble_predictions([model1, model2, model3])

# Calculate metrics on ensemble predictions
from sklearn.metrics import accuracy_score, roc_auc_score

# Convert predictions to binary predictions for accuracy
# Since these are probability outputs, we need to threshold them
threshold = 0.5
binary_preds = (avg_preds[:, 1] > threshold).float()

# Calculate accuracy
acc = accuracy_score(targets.cpu().numpy(), binary_preds.cpu().numpy())

# Calculate AUC
auc = roc_auc_score(targets.cpu().numpy(), avg_preds[:, 1].cpu().numpy())

# Calculate Gini coefficient (2*AUC-1)
gini = 2 * auc - 1

print(f"Ensemble Metrics:")
print(f"Accuracy: {acc:.6f}")
print(f"AUC Score: {auc:.6f}")
print(f"Gini Coefficient: {gini:.6f}")


Getting predictions from model 1/3
Getting predictions from model 2/3
Getting predictions from model 3/3
Ensemble Metrics:
Accuracy: 0.747500
AUC Score: 0.689549
Gini Coefficient: 0.379098


# Test Data

In [ ]:
df_test = pd.read_csv('Assessment Holdout Dataset.csv')

In [ ]:
def get_test_predictions(model, test_df):
    "Get predictions from model on test data with complete preprocessing pipeline"
    # Apply all preprocessing steps in the same order as training
    df_processed = preprocess_data(test_df)
    df_processed = feature_engineering(df_processed)
    df_processed = enhanced_feature_engineering(df_processed)
    
    # Create test dataloader
    test_dl = model.dls.test_dl(df_processed)
    
    all_preds = []
    with model.no_logging():
        for batch in test_dl:
            with torch.no_grad():
                preds = model.model(*batch)
                all_preds.append(preds)
        
        all_preds = torch.cat(all_preds)
        probs = torch.softmax(all_preds, dim=1)[:, 1]  # Use softmax since we're using CrossEntropyLoss
        pred_class = (probs > 0.5).int()
    
    return pd.DataFrame({
        'APP_ID': test_df['APP_ID'],
        'default_probability': probs.numpy(),
        'predicted_class': pred_class.numpy()
    })

In [ ]:
res = get_test_predictions(model,df_test)

In [ ]:
res

,APP_ID,default_probability,predicted_class
0,6253,0.382,0
1,4685,0.514,1
2,1732,0.512,1
3,4743,0.055,0
4,4522,0.214,0
5,6341,0.834,1
6,577,0.183,0
7,5203,0.648,1
8,6364,0.426,0
9,440,0.532,1


In [ ]:
def predict_with_ensemble(models, test_dl):
    """
    Make predictions on test data using ensemble of models
    
    Args:
        models: List of trained fastai Learner objects
        test_dl: DataLoader containing test data
    
    Returns:
        DataFrame with predictions
    """
    print("Getting ensemble predictions for test data...")
    all_preds = []
    
    # Get predictions from each model
    for i, model in enumerate(models):
        print(f"Getting predictions from model {i+1}/{len(models)}")
        # Use get_preds with the test dataloader
        test_preds, _ = model.get_preds(dl=test_dl)
        all_preds.append(test_preds)
    
    # Average predictions
    avg_preds = torch.stack(all_preds).mean(0)
    
    # Convert to probabilities if needed
    probs = avg_preds[:, 1]  # Probability of positive class
    
    # Create DataFrame with predictions
    import pandas as pd
    results = pd.DataFrame({
        'probability': probs.cpu().numpy(),
        'prediction': (probs > 0.5).int().cpu().numpy()
    })
    
    return results

In [ ]:
res = get_test_predictions(model,df_test)

In [ ]:
res

,APP_ID,default_probability,predicted_class
0,6253,0.382,0
1,4685,0.514,1
2,1732,0.512,1
3,4743,0.055,0
4,4522,0.214,0
5,6341,0.834,1
6,577,0.183,0
7,5203,0.648,1
8,6364,0.426,0
9,440,0.532,1


In [ ]:
#export res file
res.to_csv('submission.csv', index=False)

In [ ]:
def get_ensemble_test_predictions(models, test_df):
    """Get ensemble predictions from multiple models on test data"""
    # Process test data once
    df_processed = preprocess_data(test_df)
    df_processed = feature_engineering(df_processed)
    df_processed = enhanced_feature_engineering(df_processed)
    
    # Get predictions from each model manually
    all_preds = []
    for i, model in enumerate(models):
        print(f"Getting predictions from model {i+1}/{len(models)}")
        
        # Create test dataloader
        test_dl = model.dls.test_dl(df_processed)
        
        # Get predictions without using get_preds
        model_preds = []
        with torch.no_grad():
            model.model.eval()
            for batch in test_dl:
                preds = model.model(*batch)
                model_preds.append(preds)
        
        # Concatenate all batch predictions
        model_preds = torch.cat(model_preds)
        all_preds.append(model_preds)
    
    # Average predictions
    avg_preds = torch.stack(all_preds).mean(0)
    probs = torch.softmax(avg_preds, dim=1)[:, 1]  # Probability of positive class
    pred_class = (probs > 0.5).int()
    
    return pd.DataFrame({
        'APP_ID': test_df['APP_ID'],
        'default_probability': probs.cpu().numpy(),
        'predicted_class': pred_class.cpu().numpy()
    })


In [ ]:

ensemble_results = get_ensemble_test_predictions([model1, model2, model3], df_test)
ensemble_results.to_csv('ensemble_submission.csv', index=False)

Getting predictions from model 1/3
Getting predictions from model 2/3
Getting predictions from model 3/3


In [ ]:
ensemble_results

,APP_ID,default_probability,predicted_class
0,6253,0.611,1
1,4685,0.602,1
2,1732,0.587,1
3,4743,0.159,0
4,4522,0.431,0
5,6341,0.696,1
6,577,0.235,0
7,5203,0.620,1
8,6364,0.459,0
9,440,0.646,1


In [ ]:
np.clip(0.82, 0.0001, 0.9999)

np.float64(0.82)

In [ ]:
def convert_to_score(prob):
    "Convert default probability to credit score using logarithmic scaling"
    # Handle edge cases to avoid log(0) or log(infinity)
    prob = np.clip(prob, 0.0001, 0.9999)
    # Calculate odds (p/(1-p))
    odds = prob / (1 - prob)
    # Apply scaling where score of 200 = 50% probability
    # and probability halves every 100 points
    return 200 - 100 * np.log2(odds)

In [ ]:

ensemble_results['credit_score'] = convert_to_score(ensemble_results['default_probability'])

In [ ]:
ensemble_results

,APP_ID,default_probability,predicted_class,credit_score
0,6253,0.611,1,134.587
1,4685,0.602,1,140.050
2,1732,0.587,1,149.355
3,4743,0.159,0,440.199
4,4522,0.431,0,240.145
5,6341,0.696,1,80.500
6,577,0.235,0,369.910
7,5203,0.620,1,129.374
8,6364,0.459,0,223.914
9,440,0.646,1,113.222


In [ ]:
pd.read_csv('Assessment Champion Model Dataset.csv')

,APP_ID,Champion_Model_PD,Champion_Model_Score
0,6253,0.137,465
1,4685,0.152,448
2,1732,0.049,626
3,4743,0.040,657
4,4522,0.110,500
5,6341,0.122,484
6,577,0.043,646
7,5203,0.303,319
8,6364,0.124,482
9,440,0.128,477


In [ ]:
champion_df = pd.read_csv('Assessment Champion Model Dataset.csv')

# Merge with your ensemble results for comparison
comparison = ensemble_results.merge(champion_df, on='APP_ID', how='inner')

# Calculate differences between your model and champion model
comparison['pd_diff'] = comparison['default_probability'] - comparison['Champion_Model_PD']
comparison['score_diff'] = comparison['credit_score'] - comparison['Champion_Model_Score']

# Calculate performance metrics
comparison['pd_abs_diff'] = abs(comparison['pd_diff'])
comparison['score_abs_diff'] = abs(comparison['score_diff'])

# Display the comparison
comparison

,APP_ID,default_probability,predicted_class,credit_score,Champion_Model_PD,Champion_Model_Score,pd_diff,score_diff,pd_abs_diff,score_abs_diff
0,6253,0.611,1,134.587,0.137,465,0.475,-330.413,0.475,330.413
1,4685,0.602,1,140.050,0.152,448,0.451,-307.950,0.451,307.950
2,1732,0.587,1,149.355,0.049,626,0.537,-476.645,0.537,476.645
3,4743,0.159,0,440.199,0.040,657,0.119,-216.801,0.119,216.801
4,4522,0.431,0,240.145,0.110,500,0.320,-259.855,0.320,259.855
5,6341,0.696,1,80.500,0.122,484,0.574,-403.500,0.574,403.500
6,577,0.235,0,369.910,0.043,646,0.192,-276.090,0.192,276.090
7,5203,0.620,1,129.374,0.303,319,0.317,-189.626,0.317,189.626
8,6364,0.459,0,223.914,0.124,482,0.335,-258.086,0.335,258.086
9,440,0.646,1,113.222,0.128,477,0.518,-363.778,0.518,363.778


In [ ]:
cd  static

[Errno 2] No such file or directory: 'static'
/app/data/static


In [ ]:
df

,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose_cat,addr_state,debt_to_income,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,mths_since_last_major_derog,policy_code,TARGET,has_mths_since_last_delinq,has_mths_since_last_record,years_since_first_cr,pub_rec_cat,log_revol_bal,log_annual_inc,debt_income_ratio,util_acc_ratio,dti_util,delinq_2yrs_cat,inq_last_6mths_cat,mths_since_last_record_cat
0,9,MORTGAGE,45000.000,not verified,n,credit card,IL,13.950,0.000,1.000,0.000,0.000,7.000,0.000,10891,26.900,17.000,f,1,PC2,0,0,0,9,0.000,9.296,10.714,0.242,3.843,375.255,0.000,1.000,0.000
1,2,RENT,51000.000,VERIFIED - income source,n,debt consolidation,WV,10.540,0.000,0.000,44.000,0.000,6.000,0.000,6342,64.700,17.000,f,1,PC1,0,1,0,12,0.000,8.755,10.840,0.124,10.783,681.938,0.000,0.000,0.000
2,10,MORTGAGE,75000.000,VERIFIED - income,n,debt consolidation,NY,9.380,0.000,2.000,0.000,0.000,16.000,0.000,69461,47.300,37.000,f,2,PC5,1,0,0,11,0.000,11.149,11.225,0.926,2.956,443.674,0.000,2.000,0.000
3,3,RENT,84000.000,VERIFIED - income source,n,major purchase,CA,24.060,0.000,2.000,46.000,0.000,5.000,0.000,422,28.100,7.000,f,2,PC2,1,1,0,5,0.000,6.047,11.339,0.005,5.620,676.086,0.000,2.000,0.000
4,1,MORTGAGE,50000.000,not verified,n,debt consolidation,CT,6.720,1.000,1.000,22.000,0.000,7.000,0.000,11992,48.900,28.000,f,3,PC2,0,1,0,7,0.000,9.392,10.820,0.240,6.986,328.608,1.000,1.000,0.000
5,5,RENT,41900.000,not verified,n,credit card,NY,12.260,0.000,3.000,0.000,0.000,12.000,0.000,10044,42.600,15.000,f,2,PC4,1,0,0,2,0.000,9.215,10.643,0.240,3.550,522.276,0.000,3.000,0.000
6,10,MORTGAGE,99996.000,not verified,n,car small business,TX,12.840,0.000,0.000,0.000,0.000,7.000,0.000,13742,96.500,15.000,f,1,PC5,1,0,0,24,0.000,9.528,11.513,0.137,13.786,1239.060,0.000,0.000,0.000
7,5,MORTGAGE,214992.000,VERIFIED - income,n,debt consolidation small business,IL,5.740,0.000,5.000,0.000,0.000,9.000,0.000,18880,23.200,28.000,f,1,PC4,1,0,0,15,0.000,9.846,12.278,0.088,2.578,133.168,0.000,5.000,0.000
8,7,RENT,75000.000,not verified,n,debt consolidation,TX,17.580,0.000,1.000,0.000,0.000,10.000,0.000,630,2.200,15.000,f,2,PC1,0,0,0,3,0.000,6.447,11.225,0.008,0.220,38.676,0.000,1.000,0.000
9,1,RENT,32400.000,VERIFIED - income source,n,small business,OH,13.410,0.000,3.000,0.000,0.000,6.000,0.000,6514,57.100,10.000,f,3,PC5,0,0,0,11,0.000,8.782,10.386,0.201,9.517,765.711,0.000,3.000,0.000


What is a partial dependence plot? Can you explain it intuitively with a simple example? We can look at the previous example later, but first, let me explain the concept intuitively. Let's use a simple example to help understand this.


##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

A partial dependence plot (PDP) shows how a specific feature affects the predictions of a machine learning model, while accounting for the average effect of all other features.

Think of it this way: imagine you have a model that predicts house prices based on multiple features like square footage, number of bedrooms, neighborhood, and age of the house. You want to understand specifically how "square footage" impacts the predicted price, independent of other factors.

Here's how a PDP works intuitively:

1. For each possible value of square footage (say from 500 to 5000 sq ft):
   - Take all houses in your dataset
   - Temporarily set their square footage to this specific value (e.g., 1500 sq ft)
   - Keep all other features (bedrooms, neighborhood, age) at their original values
   - Get predictions for all these modified houses
   - Calculate the average predicted price

2. Plot these average predictions against the square footage values

The resulting curve shows how square footage alone influences the predicted price, averaged across all combinations of other features. For example, the plot might show that:
- Increasing from 1000 to 2000 sq ft raises the predicted price by $100,000 on average
- Beyond 3000 sq ft, additional space has diminishing returns on price

PDPs help you understand non-linear relationships and identify thresholds where a feature's impact changes significantly. They're particularly useful for interpreting complex models like random forests or gradient boosting models where the relationships between features and predictions aren't immediately obvious.

some more  example please and code snippet too...

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Let's look at a more concrete example with code. I'll show you how to create a partial dependence plot for a model that predicts diabetes progression based on patient features.

First, let's load a dataset, train a model, and create a partial dependence plot:

```python
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import partial_dependence, plot_partial_dependence
from sklearn.model_selection import train_test_split

# Load diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

# Train a random forest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Create partial dependence plot for BMI and blood pressure (BP)
features_to_plot = ['bmi', 'bp']
fig, ax = plt.subplots(figsize=(10, 6))
plot_partial_dependence(model, X_train, features_to_plot, ax=ax)
plt.tight_layout()
plt.show()
```

Now, if you want to calculate the partial dependence values manually to better understand what's happening:

```python
def calculate_pdp(model, X, feature_name, feature_values):
    """Calculate partial dependence values for a specific feature"""
    X_temp = X.copy()
    mean_predictions = []
    
    for value in feature_values:
        X_temp[feature_name] = value
        predictions = model.predict(X_temp)
        mean_predictions.append(predictions.mean())
        
    return mean_predictions

# Generate feature values for BMI
bmi_values = np.linspace(X['bmi'].min(), X['bmi'].max(), 20)
pdp_bmi = calculate_pdp(model, X_test, 'bmi', bmi_values)

# Plot manually calculated PDP
plt.figure(figsize=(8, 5))
plt.plot(bmi_values, pdp_bmi)
plt.xlabel('BMI')
plt.ylabel('Partial Dependence')
plt.title('Partial Dependence Plot for BMI')
plt.grid(True, alpha=0.3)
```

Another example with a classification task:

```python
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.inspection import plot_partial_dependence

# Load breast cancer dataset
cancer = load_breast_cancer()
X_cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
y_cancer = cancer.target

# Train a gradient boosting classifier
X_train, X_test, y_train, y_test = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42)
clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Create partial dependence plots for important features
important_features = ['mean radius', 'mean texture', 'mean perimeter']
fig, ax = plt.subplots(figsize=(12, 4))
plot_partial_dependence(clf, X_train, important_features, ax=ax)
plt.tight_layout()
plt.show()
```

These plots would help you understand how each feature affects the model's predictions independently of other features. For example, you might see that as BMI increases, the predicted diabetes progression increases steadily until a certain point, after which it plateaus. This gives you insights into the model's behavior that are difficult to extract directly from complex models.

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import partial_dependence, plot_partial_dependence
from sklearn.model_selection import train_test_split

# Load diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

# Train a random forest model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Create partial dependence plot for BMI and blood pressure (BP)
features_to_plot = ['bmi', 'bp']
fig, ax = plt.subplots(figsize=(10, 6))
plot_partial_dependence(model, X_train, features_to_plot, ax=ax)
plt.tight_layout()
plt.show()


---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
Cell In[1], line 4
      2 from sklearn.datasets import load_diabetes
      3 from sklearn.ensemble import RandomForestRegressor
----> 4 from sklearn.inspection import partial_dependence, plot_partial_dependence
      5 from sklearn.model_selection import train_test_split
      7 # Load diabetes dataset

ImportError: cannot import name 'plot_partial_dependence' from 'sklearn.inspection' (/app/data/.local/lib/python3.11/site-packages/sklearn/inspection/__init__.py)


ImportError: cannot import name 'plot_partial_dependence' from 'sklearn.inspection' (/app/data/.local/lib/python3.11/site-packages/sklearn/inspection/__init__.py)